In [3]:
from PIL import Image, ImageOps
import os
import math

# Note: run visulize_tsne_points.py first
# Folder containing individual images
image_folder = './trafficgen/img'
image_files = sorted(os.listdir(image_folder), key=lambda x: int(x.split('.')[0]))

# Assuming all images have the same size
img = Image.open(os.path.join(image_folder, image_files[0]))
img_width, img_height = img.size

new_width, new_height = 128, 128
border_size = 1
adjusted_width = new_width - 2 * border_size
adjusted_height = new_height - 2 * border_size

# Calculate grid size
grid_size = math.ceil(math.sqrt(len(image_files)))
sprite_dim = grid_size * new_width

# Create an empty sprite image
sprite_image = Image.new('RGB', (sprite_dim, sprite_dim), color='white')

for index, file in enumerate(image_files):
    img = Image.open(os.path.join(image_folder, file))
    img = img.resize((adjusted_width, adjusted_height), Image.LANCZOS)
    
    img_with_border = ImageOps.expand(img, border=border_size, fill='black')

    
    x = (index % grid_size) * new_width
    y = (index // grid_size) * new_height
    sprite_image.paste(img_with_border, (x, y))

sprite_image.save('sprite_image.png')

In [1]:
import tensorflow as tf
from tensorboard.plugins import projector
import numpy as np
import os

# Load your embeddings and metadata
embeddings = np.loadtxt('embeddings.tsv', delimiter='\t')
metadata_file = 'metadata.tsv'  # Path to your metadata.tsv

log_dir = 'log/emb'

weights = tf.Variable(embeddings, name='embedding')
checkpoint = tf.train.Checkpoint(embedding=weights)
checkpoint.save(os.path.join(log_dir, "embedding.ckpt"))

config = projector.ProjectorConfig()
embedding = config.embeddings.add()
embedding.tensor_name = 'embedding/.ATTRIBUTES/VARIABLE_VALUE'
embedding.metadata_path = 'metadata.tsv'
embedding.sprite.image_path = 'sprite_image.png'
embedding.sprite.single_image_dim.extend([128, 128])
projector.visualize_embeddings(log_dir, config)

In [2]:
%reload_ext tensorboard
%tensorboard --logdir log/emb/